# Search Space and Decorator

This tutorial explains the supported search spaces and how to use them, including
simple search spaces (Int, Real and Categorical) and nested search spaces
(Categorical, List, Dict).
AutoGluon also enables search spaces in user-defined objects using the decorator
`ag.obj` and user-defined functions using the decorator `ag.func`.

## Search Space

### Simple Search Space

In [1]:
import autogluon as ag

#### Integer Space :class:`autogluon.space.Int`

An integer will be chosen between lower and upper value during the
searcher sampleing.

In [2]:
a = ag.space.Int(lower=0, upper=10)
print(a)

Int: lower=0, upper=10


Get default value:

In [3]:
a.default

5

Change default value, which is the first configuration that a random searcher
:class:`autogluon.searcher.RandomSearcher` will try:

In [4]:
a = ag.space.Int(lower=0, upper=10, default=2)
print(a.default)

2


Pick a random value.

In [5]:
a.rand

1

#### Real Space :class:`autogluon.space.Real`

An real number will be chosen between lower and upper value during the
searcher sampleing.

In [6]:
b = ag.space.Real(lower=1e-4, upper=1e-2)
print(b)

Real: lower=0.0001, upper=0.01


Real space in log scale:

In [7]:
c = ag.space.Real(lower=1e-4, upper=1e-2, log=True)
print(c)

Real: lower=0.0001, upper=0.01


#### Categorical Space :class:`autogluon.space.Categorical`

Categorical Space will chooce one choice from all the possible values during
the searcher sampling.

In [8]:
d = ag.space.Categorical('Monday', 'Tuesday', 'Wednesday')
print(d)

Categorical['Monday', 'Tuesday', 'Wednesday']


### Nested Search Space

#### Categorical Space :class:`autogluon.space.Categorical`

Categorical Space can also be used as a nested search space.
See example at NestedExampleObj_.


#### List Space :class:`autogluon.space.List`

List Space returns a list of sampled results.

In this example, the first element of the list is a Int Space sampled
from 0 to 3, and the second element is a Categorical Space sampled
from the choices of `'alpha'` and `'beta'`.

In [9]:
f = ag.space.List(
        ag.space.Int(0, 3),
        ag.space.Categorical('alpha', 'beta'),
    )
print(f)

List[Int: lower=0, upper=3, Categorical['alpha', 'beta']]


Get one example configuration:

In [10]:
f.rand

[3, 'alpha']

#### Dict Space :class:`autogluon.space.Dict`

Dict Space returns a dict of sampled results.

Similar to List Space, the resulting configuraton of Dict is
a dict. In this example, the value of `'key1'` is sampled from
a Categorical Space with the choices of `'alpha'` and `'beta'`,
and the value of `'key2'` is sampled from an Int Space between
0 and 3.

In [11]:
g = ag.space.Dict(
        key1=ag.space.Categorical('alpha', 'beta'),
        key2=ag.space.Int(0, 3),
        key3='constant'
    )
print(g)

Dict{'key1': Categorical['alpha', 'beta'], 'key2': Int: lower=0, upper=3, 'key3': 'constant'}


Get one example configuration:

In [12]:
g.rand

{'key1': 'beta', 'key2': 0, 'key3': 'constant'}

### Decorators for Searchbale Object and Customized Training Scripts

In this section, we show how to insert search space into customized objects and
training functions.

#### Searchable space in customized class :func:`autogluon.obj`

In AutoGluon searchable object can be returned by a user defined class with a decorator.

In [13]:
@ag.obj(
    name=ag.space.Categorical('auto', 'gluon'),
    static_value=10,
    rank=ag.space.Int(2, 5),
)
class MyObj:
    def __init__(self, name, rank, static_value):
        self.name = name
        self.rank = rank
        self.static_value = static_value
    def __repr__(self):
        repr = 'MyObj -- name: {}, rank: {}, static_value: {}'.format(
                self.name, self.rank, self.static_value)
        return repr
h = MyObj()
print(h)

AutoGluonObject -- MyObj


Get one example random object:

In [14]:
h.rand

MyObj -- name: gluon, rank: 3, static_value: 10

.. _NestedExampleObj:

We can also use it within a Nested Space such as :class:`autogluon.space.Categorical`.
In this example, the resulting nested space will be sampled from

In [15]:
nested = ag.space.Categorical(
        ag.space.Dict(
                obj1='1',
                obj2=ag.space.Categorical('a', 'b'),
            ),
        MyObj(),
    )

print(nested)

Categorical[Dict{'obj1': '1', 'obj2': Categorical['a', 'b']}, AutoGluonObject -- MyObj]


Get an example output:

In [16]:
for _ in range(5):
    result = nested.rand
    assert (isinstance(result, dict) and result['obj2'] in ['a', 'b']) or hasattr(result, 'name')
    print(result)

MyObj -- name: gluon, rank: 4, static_value: 10
{'obj1': '1', 'obj2': 'a'}
MyObj -- name: auto, rank: 4, static_value: 10
{'obj1': '1', 'obj2': 'b'}
{'obj1': '1', 'obj2': 'a'}


#### Searchable space in customized function :func:`autogluon.obj`

We can also insert searchable space in a customized function:

In [17]:
@ag.func(
    framework=ag.space.Categorical('mxnet', 'pytorch'),
)
def myfunc(framework):
    return framework
i = myfunc()
print(i)

AutoGluonObject


We can also make them inside a nested space:

In [18]:
j =ag.space.Dict(
        a=ag.Real(0, 10),
        obj1=MyObj(),
        obj2=myfunc(),
    ),
print(j)

(Dict{'a': Real: lower=0, upper=10, 'obj1': AutoGluonObject -- MyObj, 'obj2': AutoGluonObject},)


#### Customized Train Script using :func:`autogluon.args`

Train_func is where to put your model training script, which takes in various keyword `args` as its hyperparameters and reports the performance of the trained model using the provided `reporter`. Here, we show a dummy train_func that simply prints these objects.

In [19]:
@ag.args(
    a=ag.space.Int(1, 10),
    b=ag.space.Real(1e-3, 1e-2),
    c=ag.space.Real(1e-3, 1e-2, log=True),
    d=ag.space.Categorical('a', 'b', 'c', 'd'),
    e=ag.space.Bool(),
    f=ag.space.List(
            ag.space.Int(1, 2),
            ag.space.Categorical(4, 5),
        ),
    g=ag.space.Dict(
            a=ag.Real(0, 10),
            obj=MyObj(),
        ),
    h=ag.space.Categorical('test', MyObj()),
    i = myfunc(),
)
def train_fn(args, reporter):
    print('args: {}'.format(args))

## Create Searcher and Sample A Configuration

#### Create a searcher and sample configuration.

In [20]:
searcher = ag.searcher.RandomSearcher(train_fn.cs)
config = searcher.get_config()
print(config)

{'a': 7, 'b': 0.005761854804661527, 'c': 0.009150367971074462, 'd.choice': 1, 'e': 0, 'f.0': 2, 'f.1.choice': 1, 'g.a': 0.9288773976854581, 'g.obj.name.choice': 0, 'g.obj.rank': 5, 'h.1.name.choice': 0, 'h.1.rank': 5, 'h.choice': 0, 'i.framework.choice': 0}


#### Run one training job with the sampled configuration:

In [21]:
train_fn(train_fn.args, config)

args: {'a': 7, 'b': 0.005761854804661527, 'c': 0.009150367971074462, 'd': 'b', 'e': 0, 'f': [2, 5], 'g': {'a': 0.9288773976854581, 'obj': MyObj -- name: auto, rank: 5, static_value: 10}, 'h': 'test', 'i': 'mxnet', '_default_config': {}}


Exit AutoGluon:

In [22]:
ag.done()